# Week 06 SVM Assignment

- Course: Z604 Music Data Mining 
- Instructor: Kahyun Choi 

Use the same Deezer MSD Mood Dataset that you used for the previous assignments. 

Links to the dataset
- Deezer MSD Mood Dataset I with three one dimensional features (moodmsdfeatures.csv): https://drive.google.com/open?id=1CHuOsBQbVCNKHu049P2u-vvd5FwRyAVs
- MSD Mood Dataset II (with actual h5 files) : https://drive.google.com/open?id=1ENp-hW4Mg_A3kW61EFZLuvfUWMnQ6JrN
- beat-aligned features from MSD h5 files (beat_aligned_feats.py): https://drive.google.com/open?id=1ElA54DvY4-gjbBdkqLWK_Zi9wlhUcmWi
- hdf5_getters.py: https://drive.google.com/open?id=1XhsIUwJhqLna85nfS2LJV9EJxU3yFMay

In [ ]:
# we will be reusing dataset from W04
import os
if os.path.exists('W04'): 
    !rm -fr 'W04/'
!git clone https://github.com/music-data-mining/W04.git
%cd W04

Cloning into 'W04'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 245 (delta 6), reused 10 (delta 4), pack-reused 233
Receiving objects: 100% (245/245), 42.97 MiB | 17.85 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/W04


# Setup

In [ ]:
# Common imports
import pandas as pd
import numpy as np
from scipy import stats
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### Load data and generate features

In [ ]:
data = pd.read_csv('moodmsdfeatures.csv')  # load data set

In [ ]:
import csv
import hdf5_getters
import beat_aligned_feats as baf

songreader = csv.DictReader(open('moodmsdfeatures.csv'))
# init segment features containers
seg_timbre_mean = np.empty((0,12), dtype=float)
seg_timbre_var = np.empty((0,12), dtype=float)
seg_chroma_mean = np.empty((0,12), dtype=float)
seg_chroma_var = np.empty((0,12), dtype=float)
seg_loudmax_mean = np.empty((0,1), dtype=float)
seg_loudmax_var = np.empty((0,1), dtype=float)
# init beat features containers
bt_timbre_mean = np.empty((0,12), dtype=float)
bt_timbre_var = np.empty((0,12), dtype=float)
bt_chroma_mean = np.empty((0,12), dtype=float)
bt_chroma_var = np.empty((0,12), dtype=float)
bt_loudmax_mean = np.empty((0,1), dtype=float)
bt_loudmax_var = np.empty((0,1), dtype=float)

for song in songreader:
    trid = song['MSD_track_id']
    filename = 'deezer_MSD/' + trid + '.h5'
    h5 = hdf5_getters.open_h5_file_read(filename)
    # extract segment features
    seg_timbre = hdf5_getters.get_segments_timbre(h5)
    seg_timbre_mean = np.vstack((seg_timbre_mean, np.mean(seg_timbre, axis = 0)))
    seg_timbre_var = np.vstack((seg_timbre_var, np.var(seg_timbre, axis = 0)))

    seg_chroma = hdf5_getters.get_segments_pitches(h5)
    seg_chroma_mean = np.vstack((seg_chroma_mean, np.mean(seg_chroma, axis = 0)))
    seg_chroma_var = np.vstack((seg_chroma_var, np.var(seg_chroma, axis = 0)))
    
    seg_loudmax = hdf5_getters.get_segments_loudness_max(h5)
    seg_loudmax_mean = np.vstack((seg_loudmax_mean, np.mean(seg_loudmax, axis = 0)))
    seg_loudmax_var = np.vstack((seg_loudmax_var, np.var(seg_loudmax, axis = 0)))    
    # extract beat features
    bt_timbre = baf.get_bttimbre(filename).T
    bt_timbre_mean = np.vstack((bt_timbre_mean, np.mean(bt_timbre, axis = 0)))
    bt_timbre_var = np.vstack((bt_timbre_var, np.var(bt_timbre, axis = 0)))
    
    bt_chroma = baf.get_btchromas(filename).T
    bt_chroma_mean = np.vstack((bt_chroma_mean, np.mean(bt_chroma, axis = 0)))
    bt_chroma_var = np.vstack((bt_chroma_var, np.var(bt_chroma, axis = 0)))    
    
    bt_loudmax = baf.get_btloudnessmax(filename)
    bt_loudmax = np.transpose(bt_loudmax)
    bt_loudmax_mean = np.vstack((bt_loudmax_mean, np.mean(bt_loudmax, axis = 0)))
    bt_loudmax_var = np.vstack((bt_loudmax_var, np.var(bt_loudmax, axis = 0)))  

### Softmax Regression using 1) segment means and vars of timbre, chroma, and loudmax and 2) beat means and vars of of timbre, chroma, and loudmax

* Build a softmax regression classifier to solve the music mood classification problem. The four mood classes are four quadrants of Russell’s emotion circumplex.
 
* Use 10% of the dataset for the test set. Report classification scores of the training set and the test set. 


* Use the following features:
seg_timbre_mean, seg_timbre_var, seg_chroma_mean, seg_chroma_var, seg_loudmax_mean, seg_loudmax_var, bt_timbre_mean, bt_timbre_var, bt_chroma_mean, bt_chroma_var, bt_loudmax_mean, bt_loudmax_var

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

X = np.hstack((seg_timbre_mean,
               seg_timbre_var,
               seg_chroma_mean,
               seg_chroma_var,
               seg_loudmax_mean,
               seg_loudmax_var,
               bt_timbre_mean,
               bt_timbre_var,
               bt_chroma_mean,
               bt_chroma_var,
               bt_loudmax_mean,
               bt_loudmax_var))
y = data['quadrant'].values.reshape(-1,)

X = preprocessing.scale(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# we use a logistic regression model, in the assignment, we will be using a SVM classifier
reg = SGDClassifier(loss="log", max_iter = 1e9)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

score = reg.score(X_train, y_train)
print("Train accuracy score: %.4f" % score)
score = reg.score(X_test, y_test)
print("Test accuracy score: %.4f" % score)

Train accuracy score: 0.7164
Test accuracy score: 0.4783


##Q1. Build a linear SVM classifier using the features you've generated (1pt)
- Split the dataset into a train and a test set (pick any test_size value you want)
- Use any hyperparameter (C or alpha) for the classifier
- Use any evaluation metric 
- Report evaluation scores of the train and the test sets



SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.001, fit_intercept=True, l1_ratio=0.15,
              learning_rate='constant', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=-inf, validation_fraction=0.1,
              verbose=0, warm_start=False)

0.6346153846153846

0.4852941176470588

## Q2. Pick the best hyperparameter for your linear SVM classifier using GridSearchCV or RandomizedSearchCV (1pt)
- Report evaluation scores of the train and the test sets
- It's totally fine even if the best parameter doesn't lead to a better evaluation score of the test set


Best CV params {'alpha': 0.01}


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='constant', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=-inf,
              validation_fraction=0.1, verbose=0, warm_start=False)

0.8333333333333334

0.4264705882352941

## Q3. Build a nonlinear SVM classifier using the features you've generated, and pick the best hyperparameter for the classifier (1pt)
- Use the RBF kernel
- Split the dataset into a train and a test set (select any test_size value you want)
- Use any hyperparameter value, C, for the classifier
- Use any evaluation metric 
- Report evaluation scores of the train and the test sets
- It's totally fine even if the best parameter doesn't lead to a better evaluation score of the test set

Best CV params {'C': 10, 'gamma': 0.001}


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

0.8333333333333334

0.4264705882352941